Creates a lazy xarray from a STAC collection and saves the result to a COG file

In [ ]:
!pip install boto3 rasterio stackstac pystac pystac_client numpy rioxarray python-dotenv

In [23]:
from boto3.session import Session
from rasterio.session import AWSSession
import stackstac
import pystac
import os
import rioxarray
from dotenv import load_dotenv

In [24]:
%load_ext dotenv
%dotenv raw_stacstack_from_cos.credentials
%dotenv raw_stacstack_from_cos.env

aws_access_key_id=os.environ.get('aws_access_key_id')
aws_secret_access_key=os.environ.get('aws_secret_access_key')
cos_endpoint_url=os.environ.get('cos_endpoint_url')
stac_endpoint=os.environ.get('stac_endpoint')
collection_id=os.environ.get('collection_id')

# cog file target location (path/)filename.cog
save_to=os.environ.get('save_to',"cogfile.cog")

# limit number of returned stac items
limit=int(os.environ.get('limit', '2'))

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [25]:
limit = 1

In [26]:
from pystac_client import Client

client = Client.open(stac_endpoint)

s1_coll = client.get_collection(collection_id)

items = []

In [27]:
item = next(s1_coll.get_items(), None)

i = 0

while item is not None:
    items.append(item.to_dict())
    i = i + 1
    if i >= limit:
        break
    item = next(s1_coll.get_items(), None)

In [28]:
session = Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

In [29]:
aws_session = AWSSession(
    session=session,
    endpoint_url=cos_endpoint_url,
)

In [30]:
data_array = stackstac.stack(items, epsg=4326, resolution=0.000089831528412, gdal_env=stackstac.DEFAULT_GDAL_ENV.updated(always=dict(session=aws_session)))

/home/romeokienzler/gitco/tensorlakehouse-openeo-driver/venv310/lib64/python3.10/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [31]:
data_array

<xarray.DataArray 'stackstac-188bf7ae9f9a91c69ff9c3eff8154c9e' (time: 1,
                                                                band: 1,
                                                                y: 21380,
                                                                x: 57109)> Size: 10GB
dask.array<fetch_raster_window, shape=(1, 1, 21380, 57109), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/17)
  * time             (time) datetime64[ns] 8B 2022-12-12T18:00:23
    id               (time) <U70 280B 'S1A_IW_GRDH_1SDV_20221212T180023_20221...
  * band             (band) <U4 16B 'data'
  * x                (x) float64 457kB -7.617 -7.617 -7.617 ... -2.487 -2.487
  * y                (y) float64 171kB 60.55 60.55 60.55 ... 58.63 58.63 58.63
    cube:variables   object 8B {'VH': {'type': 'data', 'unit': '', 'values': ...
    ...               ...
    datalayerId      <U2 8B 'DV'
    tile             <U70 280B 'S1A_IW_GRDH_1SDV_20221212T180023_20221212T180...
    polarization     <U2 8B 'VH'
    description      <U73 292B 'Cloud Optimized GeoTIFF (COG) file containing...
    title            <U2 8B 'VH'
    epsg             int64 8B 4326
Attributes:
    spec:        RasterSpec(epsg=4326, bounds=(-7.617444114752364, 58.6301639...
    crs:         epsg:4326
    transform:   | 0.00, 0.00,-7.62|\n| 0.00,-0.00, 60.55|\n| 0.00, 0.00, 1.00|
    resolution:  8.9831528412e-05

In [32]:
data_array[0,0,0,0]

<xarray.DataArray 'stackstac-188bf7ae9f9a91c69ff9c3eff8154c9e' ()> Size: 8B
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
Coordinates: (12/17)
    time             datetime64[ns] 8B 2022-12-12T18:00:23
    id               <U70 280B 'S1A_IW_GRDH_1SDV_20221212T180023_20221212T180...
    band             <U4 16B 'data'
    x                float64 8B -7.617
    y                float64 8B 60.55
    cube:variables   object 8B {'VH': {'type': 'data', 'unit': '', 'values': ...
    ...               ...
    datalayerId      <U2 8B 'DV'
    tile             <U70 280B 'S1A_IW_GRDH_1SDV_20221212T180023_20221212T180...
    polarization     <U2 8B 'VH'
    description      <U73 292B 'Cloud Optimized GeoTIFF (COG) file containing...
    title            <U2 8B 'VH'
    epsg             int64 8B 4326
Attributes:
    spec:        RasterSpec(epsg=4326, bounds=(-7.617444114752364, 58.6301639...
    crs:         epsg:4326
    transform:   | 0.00, 0.00,-7.62|\n| 0.00,-0.00, 60.55|\n| 0.00, 0.00, 1.00|
    resolution:  8.9831528412e-05

In [33]:
data_array[0,0,21380-1,57109-1]

<xarray.DataArray 'stackstac-188bf7ae9f9a91c69ff9c3eff8154c9e' ()> Size: 8B
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
Coordinates: (12/17)
    time             datetime64[ns] 8B 2022-12-12T18:00:23
    id               <U70 280B 'S1A_IW_GRDH_1SDV_20221212T180023_20221212T180...
    band             <U4 16B 'data'
    x                float64 8B -2.487
    y                float64 8B 58.63
    cube:variables   object 8B {'VH': {'type': 'data', 'unit': '', 'values': ...
    ...               ...
    datalayerId      <U2 8B 'DV'
    tile             <U70 280B 'S1A_IW_GRDH_1SDV_20221212T180023_20221212T180...
    polarization     <U2 8B 'VH'
    description      <U73 292B 'Cloud Optimized GeoTIFF (COG) file containing...
    title            <U2 8B 'VH'
    epsg             int64 8B 4326
Attributes:
    spec:        RasterSpec(epsg=4326, bounds=(-7.617444114752364, 58.6301639...
    crs:         epsg:4326
    transform:   | 0.00, 0.00,-7.62|\n| 0.00,-0.00, 60.55|\n| 0.00, 0.00, 1.00|
    resolution:  8.9831528412e-05

In [34]:
import time

start = time.time()

for index in range(21380):
    data_array[0,0,0,index]
    
end = time.time()
print(end-start)

8.267089605331421
